# Datos de entrenamiento, validación y prueba, y el método de **k-fold** de validación cruzada

https://es.wikipedia.org/wiki/Validaci%C3%B3n_cruzada

https://www.machinecurve.com/index.php/2021/02/03/how-to-use-k-fold-cross-validation-with-pytorch/

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import sklearn as skl
import pandas as pd
#from torchviz import make_dot
import torch.optim as optim
from collections import defaultdict
import pickle
import dill
import json
import datetime
from sklearn.model_selection import KFold
try:
    import google.colab
    from google.colab import files  
    COLAB = True
except:
    COLAB = False

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

In [ ]:
# La primera vez esto tarda un rato ya que tiene que bajar los datos de la red.
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Definimos el modelo de red neuronal. 
En este caso, una red convolucional profunda.

In [ ]:
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        ## Primera capa convolucional:
        ## construimos 32 canales usando filtros (kernels) de 3x3
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=32,            
                kernel_size=3,                                 
                padding=1,                  
            ),
            ## Aplicamos Batch Normalization como regularización
            nn.BatchNorm2d(32),  
            ## Aplicamos la función de activación               
            nn.ReLU(),    
            ## Reducimos la imagen con Max Pooling                  
            nn.MaxPool2d(kernel_size=2, stride=2),    
        )

        ## Segunda capa convolucional:
        ## construimos 64 canales usando filtros (kernels) de 3x3
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=32,              
                out_channels=64,            
                kernel_size=3,                               
                padding=0
            ),    
            nn.BatchNorm2d(64),
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2, stride=2),                
        )

        ## "Achatamos" la salida de la última capa, de 64 canales 
        ## de tamaño 6x6, transformandola en un vector de 64*6*6 elementos
        self.flatten = nn.Flatten()

        ## Después de las capas convoulucionales,
        ## agregamos algunas capas densas. La última, de 10
        ## neurnonas, es nuestra capa de salida
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64 * 6 * 6, 600),
            nn.Dropout(0.25), 
            nn.ReLU(),
            nn.Linear(600, 120),
            nn.Dropout(0.25), ## Regularizamos con dropout después de cada capa
            nn.ReLU(),
            nn.Linear(120, 10),
            nn.Dropout(0.25), 
            nn.ReLU() 
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)     
        x = self.linear_relu_stack(x)
        return x

In [ ]:
# Definimos la función de entrenamiento
def train_loop(dataloader,model,loss_fn,optimizer,verbose_each=32):  
    # Calculamos cosas utiles que necesitamos
    num_samples = len(dataloader.dataset)
    # Seteamos el modelo en modo entrenamiento. Esto sirve para activar, por ejemplo, dropout, etc. durante la fase de entrenamiento.
    model.train()
    # Pasamos el modelo la GPU si está disponible.        
    model = model.to(device)    
    # Iteramos sobre lotes (batchs)
    for batch,(X,y) in enumerate(dataloader):
        # Pasamos los tensores a la GPU si está disponible.
        X = X.to(device)
        y = y.to(device)      
        # Calculamos la predicción del modelo y la correspondiente pérdida (error)
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagamos usando el optimizador proveido.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Imprimimos el progreso cada 100 batchs
        if batch % verbose_each*len(X) == 0:
            loss   = loss.item()
            sample = batch*len(X) # Número de batch * número de muestras en cada batch
            #print(f"batch={batch} loss={loss:>7f}  muestras-procesadas:[{sample:>5d}/{num_samples:>5d}]")            
# De manera similar, definimos la función de validación y testeo
def test_loop(dataloader,model,loss_fn):
    num_samples  = 0
    num_batches  = 0
    avrg_loss    = 0
    frac_correct = 0
    # Seteamos el modelo en modo evaluacion. Esto sirve para desactivar, por ejemplo, dropout, etc. cuando no estamos en una fase de entrenamiento.
    model.eval()
    # Pasamos el modelo la GPU si está disponible.    
    model = model.to(device)    
    # Para validar, desactivamos el cálculo de gradientes.
    with torch.no_grad():
        # Iteramos sobre lotes (batches)
        for X,y in dataloader:
            # Pasamos los tensores a la GPU si está disponible.
            X = X.to(device)
            y = y.to(device)           
            # Calculamos las predicciones del modelo...
            pred = model(X)
            # y las correspondientes pérdidas (errores), los cuales vamos acumulando en un valor total.
            avrg_loss += loss_fn(pred,y).item()
            # También calculamos el número de predicciones correctas, y lo acumulamos en un total.
            num_batches += 1
            num_samples += y.size(0)
            frac_correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    # Calculamos la pérdida total y la fracción de clasificaciones correctas, y las imprimimos.
    avrg_loss    /= num_batches
    frac_correct /= num_samples
    #print(f"Test Error: \n Accuracy: {frac_correct:>0.5f}, Avg. loss: {avrg_loss:>8f} \n")
    return avrg_loss,frac_correct

Utilizando el método de K-fold, entrenamos y validamos modelos. En el proceso, vamos grabando los resultados en un dataframe de la librería pandas, y también, vamos grabando el modelo que mejor resultado de validación dió hasta el momento.

In [ ]:
# Definimos hiperparámetros de entrenamiento
init_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
learning_rate = 1e-3
batch_size = 1000
max_epochs = 25
num_k = 6
# Creamos un DataFrame de pandas para ir almacenando los valores calculados.
df = pd.DataFrame()
# Creamos una funcion de perdida
loss_fn = nn.CrossEntropyLoss()
# Creamos el generador de k-folds
kfold = KFold(n_splits=num_k,shuffle=False)
# Simulamos por tramos porque google colab se desconecta antes de que concluya para todos los valores de n en la lista.
min_valid_loss = 10000000.0
max_valid_accu = 0.0
for k,(train_ids,valid_ids) in enumerate(kfold.split(train_dataset)):
    # Creamos los dataloaders de entrenamiento y validacion
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=train_subsampler)
    valid_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=valid_subsampler)
    # Creamos el modelo
    model = ConvolutionalNeuralNet()
    # Creamos el optimizador
    #optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)
    # Entrenamos el modelo, calcualmos y almacenamos valores de metricas obtenidos
    min_valid_loss = float("inf")
    for epoch in range(max_epochs):
        train_loop(train_loader,model,loss_fn,optimizer)
        train_loss,train_accu = test_loop(train_loader,model,loss_fn)
        valid_loss,valid_accu = test_loop(valid_loader,model,loss_fn)
        print(f"k={k} epoch={epoch} train_loss={train_loss} train_accu={train_accu} valid_loss={valid_loss} valid_accu={valid_accu}")
        df = df.append({"k":k,
                        "epoch":epoch,
                        "train_loss":train_loss,
                        "train_accu":train_accu,
                        "valid_loss":valid_loss,
                        "valid_accu":valid_accu}
                        ,ignore_index=True)
        if min_valid_loss > valid_loss: # or max_valid_accu < valid_accu:
            if min_valid_loss > valid_loss:
                min_valid_loss = valid_loss
            if max_valid_accu < valid_accu:
                max_valid_accu = valid_accu
            # Guardamos los parámetros del modelo.
            model_fname = "kfold-best-model-"+init_datetime+".ptm"
            print("   Saving model_fname =",model_fname,end="")
            print(" ... DONE!")
            torch.save(model.state_dict(),model_fname)
json_fname = "kfold-simulation-results-"+init_datetime+".json"
df.to_json(json_fname)
if COLAB:
    files.download(model_fname)
    files.download(json_fname)

Cargamos el dataframe de pandas con los resultados de los cómputos.

In [ ]:
%%bash --out list_json
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.json

In [ ]:
list_json = list_json.split()
list_json

In [ ]:
df = pd.concat([pd.read_json(json_fname) for json_fname in list_json],ignore_index=True)
df

Procesamos los datos para poder graficarlos

In [ ]:
# Eliminamos la columna "k" que indexa los folds.
df1 = df.drop("k",1)
df1

In [ ]:
# Contamos cuantas muestras hay en cada metrica por cada epoca
df2 = df1.pivot_table(index=["epoch"],aggfunc="count").reset_index()
df2

In [ ]:
# Para epoca, calculamos el promedio de cada metrica sobre muestras
df3 = df1.pivot_table(index=["epoch"],aggfunc="mean").reset_index()
df3

Grafiquemos las metricas vs epocas

In [ ]:
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
colors = cm.Dark2.colors
for color in colors[:1]:
    x = df3["epoch"]
    ax = axes[0]
    ax.set_xlabel("epoch")
    ax.set_ylabel("loss")
    ax.plot(x,df3["train_loss"],label=f"train",color=color)
    ax.plot(x,df3["valid_loss"],label=f"valid",color=color,linestyle='--')
    ax.legend()
    ax = axes[1]
    ax.set_xlabel("epoch")
    ax.set_ylabel("accuracy")
    ax.plot(x,df3["train_accu"],label=f"train",color=color)
    ax.plot(x,df3["valid_accu"],label=f"test",color=color,linestyle='--')
    ax.legend()
fig.tight_layout()
plt.show()

Vemos que en $\mathsf{epoch} \approx 15$, $\mathsf{valid}$ $\mathsf{loss}$ se estabilza en $\approx 0.3$, mientras que $\mathsf{train}$ $\mathsf{loss}$ sigue bajando. Al mismo tiempo, vemos que $\mathsf{train}$_$\mathsf{accuracy}$ crece en todo el rango mientras que $\mathsf{valid}$ $\mathsf{accuracy}$ ya está estabilizado para $\mathsf{epoch} ≳ 15$. Concluimos, entonces, que entrenar el modelo por $\mathsf{epoch} \approx 15$ épocas es lo recomendable. Por otro lado, es importante resaltar que en dicho punto el modelo tiende a sobrefitear los datos de entrenamiento ya que $(\mathsf{valid}$ $\mathsf{loss}$ - $\mathsf{train}$ $\mathsf{loss})$/$\mathsf{train}$ $\mathsf{loss}$ $\approx 50\%$. Esto quiere decir que el modelo es demasiado complejo.

La última actualización del modelo óptimo corresponde a 

    k=5 epoch=13 train_loss=0.18559566855430604 train_accu=0.95972 valid_loss=0.28687887638807297 valid_accu=0.9231
        Saving model_fname = kfold-best-model-2022-02-07-21-45-00.ptm ... DONE!

ocurriendo para $\mathsf{epoch}=13$, lo cual es muy cercano al valor $\mathsf{epoch}=15$ que mencionamos anteriormente.

Probamos el modelo óptimo seleccionado por el algoritmo (correspondiente a $\mathsf{epoch}=13$) en el conjunto de validación.

In [ ]:
%%bash --out model_fname
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.ptm

In [ ]:
model_fname = model_fname.split()[0]

In [ ]:
model = ConvolutionalNeuralNet()
model.load_state_dict(torch.load(model_fname,map_location="cpu"))
model.eval()
model = model.to(device)

In [ ]:
batch_size = 1000
loss_fn = nn.CrossEntropyLoss()
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size)
test_loss,test_accu = test_loop(test_loader,model,loss_fn)
print("test_loss = ",test_loss)
print("test_accu = ",test_accu)

Por comparación:

    epoch	train_accu	train_loss	valid_accu	valid_loss
    ...
    12	   0.946237	   0.225827	  0.915500	  0.318581
    13	   0.954733	   0.190428	  0.919967	  0.290598
    14	   0.957810	   0.186052	  0.920117	  0.298289
    15	   0.958237	   0.186528	  0.918267	  0.305730
    16	   0.963387	   0.165904	  0.922633	  0.290206
    17	   0.965727	   0.159663	  0.922067	  0.291162
    ...

Concluimos así que los valores de validación son confiables, y que $\mathsf{epoch} = 15$ constituye un adecuado valor de número de épocas de entrenamiento.